In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/Molecular Machine Learning - Project


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1UIPTSI8IVCmTuaGOnbXQoz8QVibR_cvr/Molecular Machine Learning - Project


In [ ]:
%ls

 aimnet-x2d/
 dataset/
'MML project slide.gslides'
'Molecular Machine Learning - Project Proposal.gdoc'
 Peer_Review.gdoc
'Revised-Molecular ML Project Proposal.gdoc'
 Run_AIMNet-X2D.ipynb


In [ ]:
# !git clone https://github.com/isayevlab/aimnet-x2d.git

In [ ]:
%ls
%cd aimnet-x2d/

 aimnet-x2d/
 dataset/
'MML project slide.gslides'
'Molecular Machine Learning - Project Proposal.gdoc'
 Peer_Review.gdoc
'Revised-Molecular ML Project Proposal.gdoc'
 Run_AIMNet-X2D.ipynb
/content/drive/.shortcut-targets-by-id/1UIPTSI8IVCmTuaGOnbXQoz8QVibR_cvr/Molecular Machine Learning - Project/aimnet-x2d


In [ ]:
# when first time, run this and there'll be a runtime error,
# restart it and comment out this line,run above cell and skip this cell,
# run the next cell, and the runtime needs to be restart again,
# and everything should be good

# pip install -r requirements.txt


In [ ]:
!pip install --upgrade scikit-learn

# Fix tornado for google-colab
!pip install tornado==6.5.1

# Downgrade dill and multiprocess if you need the datasets library
!pip install dill==0.3.8 multiprocess==0.70.16

In [ ]:
import torch
import torch_geometric
print(f"PyTorch: {torch.__version__}")
print(f"PyG: {torch_geometric.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch: 2.5.1+cu121
PyG: 2.6.1
CUDA available: True


In [ ]:
%cd ..
%ls

/content/drive/.shortcut-targets-by-id/1UIPTSI8IVCmTuaGOnbXQoz8QVibR_cvr/Molecular Machine Learning - Project


In [ ]:
import pandas as pd

df = pd.read_csv('dataset/csv/train.csv')
print(f"Total samples: {len(df)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nMissing data:")
for col in ['Tg', 'FFV', 'Tc', 'Density', 'Rg']:
    if col in df.columns:
        missing = df[col].isna().sum()
        print(f"  {col}: {missing}/{len(df)} missing ({missing/len(df)*100:.1f}%)")

print(f"\nFirst 3 SMILES:")
print(df['SMILES'].head(3).tolist())

Total samples: 7973

Columns: ['id', 'SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']

Missing data:
  Tg: 7462/7973 missing (93.6%)
  FFV: 943/7973 missing (11.8%)
  Tc: 7236/7973 missing (90.8%)
  Density: 7360/7973 missing (92.3%)
  Rg: 7359/7973 missing (92.3%)

First 3 SMILES:
['*CC(*)c1ccccc1C(=O)OCCCCCC', '*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5ccc(N*)cc5)cc4)CCC(CCCCC)CC3)cc2)cc1', '*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(S(=O)(=O)c7ccc(Oc8ccc(C=C9CCCC(=Cc%10ccc(*)cc%10)C9=O)cc8)cc7)cc6)cc5)CCCCC4)cc3)cc2)cc1']


In [ ]:
df_clean_FFV = df.copy()
df_clean_FFV.drop(df[df['FFV'].isna()].index, inplace=True)

In [ ]:
df_clean_FFV.to_csv('dataset/csv/train_clean_FFV.csv', index=False)

In [ ]:
# Test run : Regression for FFV

!python aimnet-x2d/main.py \
  --data_path dataset/csv/train_clean_FFV.csv \
  --smiles_column SMILES \
  --task_type regression \
  --target_column FFV \
  --train_split 0.8 --val_split 0.1 --test_split 0.1 \
  --model_save_path models/test_model.pth \
  --batch_size 32 \
  --epochs 5 \
  --hidden_dim 256

AIMNET-X2D CONFIGURATION
Data Configuration:
  Single file: dataset/csv/train_clean_FFV.csv
  Train/Val/Test splits: 0.8/0.1/0.1
  SMILES column: SMILES
  Target column: FFV

Model Configuration:
  Hidden dimension: 256
  Number of shells: 3
  Message passing layers: 3
  Pooling type: attention
  Activation: silu
  Partial charges: False
  Stereochemistry: False

Training Configuration:
  Task type: regression
  Learning rate: 0.00025
  Epochs: 5
  Batch size: 32
  Loss function: l1
  Early stopping: False

System Configuration:
  GPU devices: 1
  Workers: 4
  Mixed precision: False
  Iterable dataset: False
✅ Configuration valid
SYSTEM INFORMATION
PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
GPU devices: 1
  Device 0: NVIDIA L4
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Python version: 3.12.12
CPU cores: 12
Total memory: 52.96 GB
Available memory: 50.41 GB
✅ Configuration valid
[Single-Process] Using device: cuda
RUNNING TRAINING MODE
Loading and preprocess